In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:12:34 of 0:12:35 remaining]
Chain 1:  10% [0:00:48 of 0:00:53 remaining]
Chain 1:  20% [0:00:50 of 0:01:03 remaining]
Chain 1:  30% [0:00:47 of 0:01:07 remaining]
Chain 1:  40% [0:00:41 of 0:01:09 remaining]
Chain 1:  50% [0:00:35 of 0:01:10 remaining]
Chain 1:  60% [0:00:28 of 0:01:11 remaining]
Chain 1:  70% [0:00:21 of 0:01:11 remaining]
Chain 1:  80% [0:00:14 of 0:01:11 remaining]
Chain 1:  90% [0:00:07 of 0:01:11 remaining]
Chain 1: 100% [0:00:00 of 0:01:12 remaining]

Chain 2:   0% [0:00:03 of 0:00:03 remaining]
Chain 2:  10% [0:00:55 of 0:01:01 remaining]
Chain 2:  20% [0:00:52 of 0:01:05 remaining]
Chain 2:  30% [0:00:47 of 0:01:07 remaining]
Chain 2:  40% [0:00:41 of 0:01:09 remaining]
Chain 2:  50% [0:00:34 of 0:01:07 remaining]
Chain 2:  60% [0:00:27 of 0:01:08 remaining]
Chain 2:  70% [0:00:21 of 0:01:09 remaining]
Chain 2:  80% [0:00:14 of 0:01:09 remaining]
Chain 2:  90% [0:00:07 of 0:01:10 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[28.4273 112.564 0.930925; 27.5248 112.564 0.930925; … ; 27.3239 116.748 0.845701; 30.7513 117.583 0.828856]

[27.4302 116.41 0.847305; 27.642 114.969 0.878734; … ; 26.7839 112.404 0.943431; 24.2814 112.404 0.943431]

[24.3522 113.104 0.912927; 25.687 111.484 0.959265; … ; 27.6138 116.368 0.857248; 27.3044 116.545 0.846114]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD        Naive SE        MCSE        ESS   
     s2  26.1808028 2.000867124 0.01217688954 0.01993324931 9000.0000
beta[1] 113.8239535 1.897484771 0.01154772457 0.03487090407 2960.9439
beta[2]   0.9061479 0.041741608 0.00025403133 0.00075708687 3039.8133

Quantiles:
            2.5%         25.0%       50.0%       75.0%        97.5%   
     s2  22.55114241  24.77092147  26.0849397  27.4690796  30.36704903
beta[1] 110.01767323 112.55486623 113.8382832 115.0984947 117.49059197
beta[2]   0.82547148   0.87786495   0.9060332   0.9339591   0.98996128



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*